In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
train.head() 

In [ ]:
train.describe()

In [ ]:
train.target.value_counts()

训练集20万行，有20万个样本。0.0 ~ 19 9999.0\
每行75个feature，feature_0 ~ feature_74\
每个特征只有少数样本有几十的取值 \
大部分是很小的数字（3/4分位点都是1、2这种小数）\
少部分取值极大，能到几十\
所有特征值都是浮点类型，但看起来小数部分都是0\
类别数从class_1 ~ class_9 一共9类

In [ ]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')
test.head()

In [ ]:
test.describe()

test有10万个样本，id号从20 0000 到 29 9999\
特征值也是大量小值+少量极大值

In [ ]:
train.keys()
train['feature_19'].value_counts()
# train['feature_19'].value_counts(sort=False)[:20]

In [ ]:
import matplotlib.pyplot as plt
for i,key in enumerate(train):
    # print(train[key].value_counts())
    if i == 20:
        plt.figure()
        plt.xlabel('feature value')
        plt.ylabel('count')
        plt.title(key)
        plt.plot(train[key].value_counts()[:10])


数据类型已经全为数值型\
问题目标 是输出一个样本属于9种类别的概率

In [ ]:
#看看提交数据的格式
submit = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv')
submit

# 开始处理数据

In [ ]:
X_train = train.iloc[:,1:-1].values
y_train = train.iloc[:,-1].values
X_test = test.iloc[:,1:].values
X_test

In [ ]:
X_train.shape

In [ ]:
y_train_int = np.array([int(item[-1]) for item in y_train])

In [ ]:
#对每一个类别建立一个回归分析 logistics
#根据属性值作为自变量，输出概率得分
#问题在于，训练集的y 值只有01，比如针对class01，不管属性向量X 取多少，y只有0和1，这对于回归不能用吧？
#先试试
 
#另一个按方案是，就把类标签转为数字1~9，然后拟合。得到2.5 就让class2 和 class3 各有0.5 的probability，2.9 则前者0.1 后者0.9 ？亦或者是高斯分布？

In [ ]:
from sklearn.linear_model import LinearRegression as LR

rgs = LR()
rgs.fit(X_train,y_train_int)
y_pred = rgs.predict(X_test)

In [ ]:
plt.hist(y_pred)

In [ ]:
#按照回归输出填写分类矩阵，出现在中间的把靠近的设大，出现在0 9 两边的直接给0 或 9
for id,score in enumerate(y_pred):
#     print(id)
    for i in range(1,10):
        submit.iloc[id,i] = 0
    low = int(score)
    high = low + 1
    if score<1:
        submit.iloc[id,1] = 1
        continue
    if score>9:
        submit.iloc[id,9] = 0
        continue
    submit.iloc[id,low]  = high - score 
    submit.iloc[id,high] = score - low
    # for example score = 2.8
    # then class_2=0.2 class_3=0.8

submit.head()

In [ ]:
#然后直接把这个结果上传
#先生成文件 linerRegression-01.csv
submit.to_csv('./LR-01.csv',index=False)

In [ ]:
reload = pd.read_csv('./LR-01.csv')
reload